In [1]:
import torch
import pandas as pd
import numpy as np
import pickle
import LogLikelihoodTorch
import plotly.express as px

with open('DataPhase.pickle','rb') as f:
    X = pickle.load(f) 
X_tensor = X_tensor = torch.from_numpy(X).to(torch.float64)
#X_tensor = torch.reshape(X_tensor.T,(-1,90,330))
torch.set_default_dtype(torch.float64)

In [2]:
Repetetions = 5
Subjects = 199

with open('MM_K_Eval.pickle','rb') as f:
    Likelihoods = pickle.load(f)

Xtrain = torch.zeros((90,165*Subjects))
Xtest = torch.zeros((90,165*Subjects))
for i in range(Subjects):
    Xtrain[:,i*(165):((i+1)*165)] = X_tensor[:,i*(330):((i*330+165))]
    Xtest[:,i*(165):((i+1)*165)] = X_tensor[:,i*(330)+165:((i*330+330))]

lr= 1
n_iters=3000

In [21]:
for idx,K in enumerate(range(18,21)):
    for rep in range(Repetetions):
            pi,kappa,mu = LogLikelihoodTorch.InitializeFF(Xtrain,p=90,K=K)

            Parameters = [
                {'params':pi},
                {'params':kappa},
                {'params':mu}
            ]

            Adam = torch.optim.Adam(Parameters,lr=lr)
            LogLikelihoodTorch.Optimizationloop(Xtrain,[pi,kappa,mu],lose=LogLikelihoodTorch.log_likelihood,Optimizer=Adam,n_iters=n_iters,K=K)

            Likelihoods[rep,K-2] = LogLikelihoodTorch.log_likelihood(Xtest,pi,kappa,mu,p=90,K=K)

epoch 1; Log-Likelihood = -2743035.4524181196
epoch 101; Log-Likelihood = -3243933.140883669
epoch 201; Log-Likelihood = -3273473.8984689466
epoch 301; Log-Likelihood = -3318928.3576388564
epoch 401; Log-Likelihood = -3361338.6002594526
epoch 501; Log-Likelihood = -3398111.41486667
epoch 601; Log-Likelihood = -3441470.710760632
epoch 701; Log-Likelihood = -3495159.1594812325
epoch 801; Log-Likelihood = -3543828.0529320417
epoch 901; Log-Likelihood = -3596067.054496192
epoch 1001; Log-Likelihood = -3651627.509543604
epoch 1101; Log-Likelihood = -3710191.685469975
epoch 1201; Log-Likelihood = -3771416.6851946847
epoch 1301; Log-Likelihood = -3823404.8111053496
epoch 1401; Log-Likelihood = -3860329.3504264555
epoch 1501; Log-Likelihood = -3898153.7580891796
epoch 1601; Log-Likelihood = -3923100.287776556
epoch 1701; Log-Likelihood = -3940631.7985259844
epoch 1801; Log-Likelihood = -3947328.1887899027
epoch 1901; Log-Likelihood = -3953822.32545512
epoch 2001; Log-Likelihood = -3957657.9573

In [22]:
print(Likelihoods)
with open('MM_K_Eval.pickle','wb') as f:
    pickle.dump([Likelihoods], f)

[[2975718.53899757 3075621.4847755  3284261.47150799 3327782.8476927
  3415744.42464808 3515985.04419563 3630010.01885179 3653975.28775978
  3682485.59458762 3748775.86683707 3807894.67682633 3874099.24674927
  3796873.75339701 3837873.87772977 3985909.00847288 3872840.53517406
  3888124.96580141 3958974.79068914 3988340.46272673]
 [2975812.99181734 3075705.39040973 3284125.54096477 3327790.46948344
  3466489.76294985 3415817.71930862 3487478.59482024 3735631.49452169
  3699383.7949197  3753561.7084699  3796873.95660722 3859693.23027628
  3830734.29041905 3770912.99417168 3811012.68944127 3880701.70919462
  3913341.77249389 3992849.36493644 3873007.58095414]
 [2975761.41075484 3075822.99251743 3284207.72447403 3327763.7953262
  3415769.17107732 3500636.71339295 3629957.91884485 3643596.03477497
  3630020.78757094 3690254.69212157 3841137.2818878  3765004.68184735
  3810912.19329337 3881734.91780485 3943004.27405669 3945685.09242596
  3881948.48066704 4037745.93926331 4023607.80326343]


EOFError: Ran out of input